# Практика
Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import random
import time


## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [2]:
# headers={
#     'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
# }
# def get_content(url):
#     with requests.Session() as req:
#         req.headers.update(headers)
#         r = req.get(url).content
#     return r
# 
# url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
# s = get_content(url)
# df=pd.read_csv(io.StringIO(s.decode('UTF8')))


### Загрузка онлайн не работает
df=pd.read_csv("Химический_анализ_родника_в_Нескучном_саду.csv", sep=';')
display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


In [3]:
df = df.replace({'Результат анализа':{'б/цвета':0}})

def two(a, b):
  if a[:10] == 'в пределах':
    if float(a[11:12]) <= float(b) <= float(a[13:]):
        return 'Да'
    else:
        return 'Нет'
  elif a.find('не более') !=-1 and a.find('-') != -1:
    if float(b) <= float(a[11:]):
        return 'Да'
    else:
        return 'Нет'        
  else:
    a1 = a[8:]
    a1 = a1.replace(',', '.')
    if float(b) <= float(a1):
        return 'Да'
    else:
        return 'Нет' 

df['В норме'] = df.apply(lambda x: two(a = x['Норматив'], b = x['Результат анализа']), axis=1)
df = df.set_index('Показатель')
df



,Единица измерений,Результат анализа,Норматив,В норме
Показатель,,,,
pH,единицы pH,8.4,в пределах 6-9,Да
Запах,баллы,1,не более 2-3,Да
Цветность,градусы,0,не более 30,Да
Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10,Да
Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5",Да
Нитриты (по NO2),мг/дм3,0.017,"не более 3,3",Да
Нитраты (по NO3),мг/дм3,24,не более 45,Да
Фосфаты (P),мг/дм3,0.36,"не более 3,5",Да
Хлориды (Cl),мг/дм3,200,не более 350,Да


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он экспериментально?


In [4]:
%%time
box = ['orange']*5 + ['apple']*4

def one(w):
  box1 = ['orange']*5 + ['apple']*4
  box1.remove(w)
  return random.choice(box1)

def two(o, p):
  box2 = ['orange']*5 + ['apple']*4
  box2.remove(o)
  box2.remove(p)
  return random.choice(box2)

n = 1000000
# формируем датафрейм с нашими экспериментами
df = pd.DataFrame({"A":list(map(lambda a: random.choice([
                           random.choice(box)
                           ]), range(n)
                              ))
                 })

df['B'] = df['A'].map(one)

df['C'] = df.apply(lambda x: two(o = x['A'], p= x['B']), axis=1)

p = len(df[((df.A == 'orange') & (df.B == 'orange') & (df.C == 'orange'))])/df.shape[0]
print(f"{p} - вероятность события, что все три вынутых фрукта апельсины")

0.118392 - вероятность события, что все три вынутых фрукта апельсины
CPU times: total: 18.8 s
Wall time: 19.2 s


Значение вероятности события, что все три фрукта апельсины, вычисленное аналитически (0.119), экспериментально подтверждается (0.118392 при n = 1000000).

### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [9]:
%%time
box = ['nest']*3 + ['st']*7

def one(w):
  if w == 'st':
    return None
  box1 = ['nest']*3 + ['st']*7
  box1.remove(w)
  return random.choice(box1)

n = 1000000
# формируем датафрейм с нашими экспериментами
Z = pd.DataFrame({"A":list(map(lambda a: random.choice([
                           random.choice(box)
                           ]), range(n)
                              ))
                 })

Z['B'] = Z['A'].map(one)

p = len(Z[((Z.A == 'nest') & (Z.B == 'st'))])/Z.shape[0]
print(f"{p} - вероятность события, что мастер проверил только две детали")

0.233243 - вероятность события, что мастер проверил только две детали
CPU times: total: 2.27 s
Wall time: 2.3 s


Значение вероятности события, что мастер проверит ровно две детали, вычисленное аналитически (0.23333), экспериментально подтверждается (0.233243 при n = 1000000).